# Goal
1. load row data
2. convert text data to correct datatype
3. show static's about data
3. visualize row data ( understand the data we are using)

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
BASE_DIR = os.path.abspath("..")
RAW_DATA_PATH = os.path.join(BASE_DIR, "dataset")


In [ ]:
installs = pd.read_csv(os.path.join(RAW_DATA_PATH, "installs.txt"), sep="\t")
brochure_views = pd.read_csv(os.path.join(RAW_DATA_PATH, "brochure views.txt"), sep="\t")
brochure_views_july = pd.read_csv(os.path.join(RAW_DATA_PATH, "brochure views july.txt"), sep="\t")
app_starts = pd.read_csv(os.path.join(RAW_DATA_PATH, "app starts.txt"), sep="\t")
app_starts_july = pd.read_csv(os.path.join(RAW_DATA_PATH, "app starts july.txt"), sep="\t")

## convert datatypes

In [ ]:
print(installs.info())
print(brochure_views.info())
print(app_starts.info())

In [ ]:
installs['InstallDate'] = pd.to_datetime(installs['InstallDate'], errors='coerce')
brochure_views['dateCreated'] = pd.to_datetime(brochure_views['dateCreated'], errors='coerce')
brochure_views_july['dateCreated'] = pd.to_datetime(brochure_views_july['dateCreated'], errors='coerce')
app_starts['dateCreated'] = pd.to_datetime(app_starts['dateCreated'], errors='coerce')
app_starts_july['dateCreated'] = pd.to_datetime(app_starts_july['dateCreated'], errors='coerce')

## data overview



In [ ]:
print("Data Shapes:")
print("Installs:", installs.shape)
print("Brochure Views:", brochure_views.shape)
print("Brochure Views July (Test):", brochure_views_july.shape)
print("App Starts:", app_starts.shape)
print("App Starts July (Test):", app_starts_july.shape)



In [ ]:
print("\nPreview of Installs:")
display(installs.head())

print("\nPreview of Brochure Views:")
display(brochure_views.head())

print("\nPreview of App Starts:")
display(app_starts.head())


In [ ]:
print("Missing Values:")
print("Missing in Installs:\n", installs.isna().sum())
print("Missing in Brochure Views:\n", brochure_views.isna().sum())
print("Missing in App Starts:\n", app_starts.isna().sum())

### duration analysis on row data
after viewing the data on brochure_views it was observed that the view_duration had missing values and data errors with negative duration


In [ ]:
column ='view_duration'
missing_duration = brochure_views[brochure_views[column].isna()]
missing_count = missing_duration.shape[0]
print("Number of rows with missing view_duration:", missing_count)
percentage_missing = (missing_count / brochure_views.shape[0]) * 100
print("Percentage of missing view_duration:", percentage_missing)
display(missing_duration.head())
print(f"min duration value: {brochure_views[column].min()}")
# finding positive min value for replacing as view duration
print(f"min positive duration value: {brochure_views[column][brochure_views[column]>0].min()}")


In [ ]:
negative_count = (brochure_views[column] < 0).sum()
percentage_negative_duration = (negative_count / brochure_views.shape[0]) * 100
print("Number of rows with negative view_duration:", negative_count)
print("Percentage of negative view_duration:", percentage_negative_duration)

In [ ]:
plt.figure(figsize=(4,4))
sns.histplot(brochure_views['view_duration'], kde=True)
plt.title("Box Plot of Raw View Duration")
plt.tight_layout()
plt.show()

In [ ]:
# Checking page_turn_count distribution
plt.figure(figsize=(4,4))
sns.histplot(brochure_views['page_turn_count'], kde=True)
plt.title("Distribution of Page Turn Count (April-June)")
plt.xlabel("Page Turn Count")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()


In [ ]:
# view Duplicate values
print("Duplicates in installs:", installs.duplicated().sum())
print("Duplicates in brochure_views:", brochure_views.duplicated().sum())
print("Duplicates in app_starts:", app_starts.duplicated().sum())

### app_starts analysis

In [ ]:
# have a sample of duplicated values
duplicated_rows = app_starts[app_starts.duplicated(keep=False)]

print(f"Number of duplicated rows in app_starts: {duplicated_rows.shape[0]}")
display(duplicated_rows.head())

### check for inconstancy app_start


In [ ]:

app_starts.drop_duplicates(inplace=True)
# get the earliest installation
user_earliest_install = installs.groupby('userId', as_index=False)['InstallDate'].min()
user_earliest_install.rename(columns={'InstallDate': 'earliest_install_date'}, inplace=True)

app_starts_install = app_starts.merge(user_earliest_install, on='userId', how='left')
app_starts_filtered = app_starts_install[app_starts_install['dateCreated'] >= app_starts_install['earliest_install_date']]
print("Original app_starts rows:", app_starts.shape[0])
print("Filtered app_starts rows:", app_starts_filtered.shape[0])
consistent_data = (app_starts_filtered.shape[0] / app_starts.shape[0]) * 100
print("Percentage of inconsistent app_starts:", consistent_data)
display(app_starts_filtered.head())

### analyse install data

In [ ]:
installs_unique = installs.drop_duplicates(subset='userId', keep='first')
print("Number of  installs:", installs.shape[0])
print("Number of unique installs:", installs_unique.shape[0])

### static summary


In [ ]:
print("\nSummary Statistics for Installs:")
display(installs.describe())

print("\nSummary Statistics for Brochure Views (April-June):")
display(brochure_views.describe())

print("\nSummary Statistics for App Starts (April-June):")
display(app_starts.describe())

# Summary statistics for categorical columns
print("\nCategorical Data Summary - Installs:")
display(installs.describe(include=['object']))

print("\nCategorical Data Summary - Brochure Views (April-June):")
display(brochure_views.describe(include=['object']))

print("\nCategorical Data Summary - App Starts (April-June):")
display(app_starts.describe(include=['object']))